In [49]:
import pandas as pd
import numpy as np
import time

In [50]:
returns = pd.read_excel("Returns_Clean.xlsx")
#returns.index = pd.to_datetime(returns.index, format='%Y%m%d')
flows = pd.read_excel("Flows_Clean.xlsx")
#flows.index = pd.to_datetime(flows.index, format='%Y%m%d')

In [51]:
returns.head()

,Dates,55976,41072,27748,75259,40970,62519,21792,54084,55511,...,63829,44986,11891,22752,27780,59379,75162,27705,54594,48725
0,19930401,-0.019685,0.019391,-0.007282,-0.003891,0.083333,0.027027,0.005348,0.020000,-0.010417,...,0.108108,-0.019048,-0.017857,-0.028269,0.000000,0.006173,-0.023669,0.000000,0.031915,0.014433
1,19930402,-0.020080,0.002717,-0.002445,0.058594,-0.038462,-0.026316,-0.007979,-0.019608,-0.021053,...,0.000000,-0.016181,-0.031818,-0.010909,0.000000,-0.020450,0.012121,-0.032609,0.020619,-0.010163
2,19930405,-0.024590,-0.006775,-0.002451,0.022140,-0.040000,0.027027,0.005362,0.000000,0.021505,...,-0.048780,-0.003289,0.032864,0.003676,-0.005102,0.018789,-0.005988,-0.011236,-0.010101,0.026694
3,19930406,-0.067227,0.009550,-0.007371,0.025271,0.041667,0.052632,0.005333,0.000000,0.000000,...,-0.025641,-0.023102,-0.009091,-0.007326,0.005128,0.002049,-0.024096,-0.011364,-0.010204,0.014000
4,19930407,0.000000,0.005405,0.000000,0.021127,0.040000,-0.050000,0.002653,0.000000,0.010526,...,-0.013158,0.016892,0.018349,0.000000,0.010204,0.014315,0.000000,0.011494,0.000000,0.005917


In [52]:
ticker = returns.columns.tolist()[1:]

In [53]:
dates = returns.Dates
# The portfolio returns and flows are the average of all stocks
port_returns = pd.DataFrame(data = returns.iloc[:,1:].mean(axis=1), columns = ['Portfolio_returns'] )
# print(port_returns.head())
port_flows = pd.DataFrame(data = flows.iloc[:,1:].mean(axis=1), columns = ['Portfolio_flows'])
# port_flows.head()
portfolio = pd.concat([dates, port_returns, port_flows], axis = 1)
portfolio.head()

,Dates,Portfolio_returns,Portfolio_flows
0,19930401,-0.001138,-1.399531
1,19930402,-0.015585,-2.371056
2,19930405,0.003637,-0.140873
3,19930406,-0.003112,0.453289
4,19930407,-0.001031,-0.559098


In [54]:
# generate predictor matrix for portfolio
# predictor matrix P_small, includes portfolio's own returns up to three lags and portfolio’s net trading flows up to three lags

P_small = pd.DataFrame()   
for j in range(1,4):
    p_t = port_returns.iloc[:len(port_returns)-j]
    f_t = port_flows.iloc[:len(port_flows)-j]
    P_small = pd.concat([P_small, p_t], axis = 1)
    P_small = pd.concat([P_small, f_t], axis = 1)

In [55]:
def add_lag(df, lag, feature_name):   
    for i in range(lag):
        column_name = feature_name + '_lag_' + str(i+1)
        df[column_name] = df[feature_name].shift(i+1)
    return df

In [56]:
lag = 3
feature_name = 'Portfolio_flows'
flow_lag = add_lag(portfolio, lag, feature_name)
flow_lag.head()

,Dates,Portfolio_returns,Portfolio_flows,Portfolio_flows_lag_1,Portfolio_flows_lag_2,Portfolio_flows_lag_3
0,19930401,-0.001138,-1.399531,NaN,NaN,NaN
1,19930402,-0.015585,-2.371056,-1.399531,NaN,NaN
2,19930405,0.003637,-0.140873,-2.371056,-1.399531,NaN
3,19930406,-0.003112,0.453289,-0.140873,-2.371056,-1.399531
4,19930407,-0.001031,-0.559098,0.453289,-0.140873,-2.371056


In [57]:
lag = 3
feature_name = 'Portfolio_returns'
flow_ret = add_lag(portfolio, lag, feature_name)
flow_ret.head()

,Dates,Portfolio_returns,Portfolio_flows,Portfolio_flows_lag_1,Portfolio_flows_lag_2,Portfolio_flows_lag_3,Portfolio_returns_lag_1,Portfolio_returns_lag_2,Portfolio_returns_lag_3
0,19930401,-0.001138,-1.399531,NaN,NaN,NaN,NaN,NaN,NaN
1,19930402,-0.015585,-2.371056,-1.399531,NaN,NaN,-0.001138,NaN,NaN
2,19930405,0.003637,-0.140873,-2.371056,-1.399531,NaN,-0.015585,-0.001138,NaN
3,19930406,-0.003112,0.453289,-0.140873,-2.371056,-1.399531,0.003637,-0.015585,-0.001138
4,19930407,-0.001031,-0.559098,0.453289,-0.140873,-2.371056,-0.003112,0.003637,-0.015585


In [65]:
P_small = flow_ret.dropna()
#P_small.Dates = P_small.Dates.apply(lambda x: pd.to_datetime(x, format='%Y%m%d'))

In [66]:
print(P_small.shape)
P_small.head()

(1955, 9)


,Dates,Portfolio_returns,Portfolio_flows,Portfolio_flows_lag_1,Portfolio_flows_lag_2,Portfolio_flows_lag_3,Portfolio_returns_lag_1,Portfolio_returns_lag_2,Portfolio_returns_lag_3
3,19930406,-0.003112,0.453289,-0.140873,-2.371056,-1.399531,0.003637,-0.015585,-0.001138
4,19930407,-0.001031,-0.559098,0.453289,-0.140873,-2.371056,-0.003112,0.003637,-0.015585
5,19930408,-0.000309,0.160661,-0.559098,0.453289,-0.140873,-0.001031,-0.003112,0.003637
6,19930412,0.009771,1.440657,0.160661,-0.559098,0.453289,-0.000309,-0.001031,-0.003112
7,19930413,0.003260,0.773246,1.440657,0.160661,-0.559098,0.009771,-0.000309,-0.001031


In [67]:
P_small.to_csv('P_small.csv', encoding='utf-8')

# P_full

In [13]:
# Import X_small
X_small = pd.read_excel('X_small.xlsx')
X_small = X_small.drop(columns=['Unnamed: 0','flow'])
print(X_small.shape)
X_small.head()

(195500, 9)


,Dates,PERMNO,ret,ret_lag_1,ret_lag_2,ret_lag_3,flow_lag_1,flow_lag_2,flow_lag_3
0,19930406,55976,-0.067227,-0.024590,-0.020080,-0.019685,-5.642141,-6.975709,-4.696503
1,19930407,55976,0.000000,-0.067227,-0.024590,-0.020080,-6.975709,-5.642141,-6.975709
2,19930408,55976,0.040541,0.000000,-0.067227,-0.024590,-6.975709,-6.975709,-5.642141
3,19930412,55976,0.038961,0.040541,0.000000,-0.067227,-6.975709,-6.975709,-6.975709
4,19930413,55976,-0.033333,0.038961,0.040541,0.000000,-3.351631,-6.975709,-6.975709


In [45]:
P_full = P_small[['Dates','Portfolio_returns','Portfolio_returns_lag_1','Portfolio_flows_lag_1']]
start =time.process_time()

for i in ticker:
    dt = X_small[X_small.PERMNO==i][['ret_lag_1','flow_lag_1']]
    dt = pd.concat([dt],axis=0).reset_index(drop=True)
    dt.columns = ['ret_lag1_'+str(i),'flow_lag1_'+str(i)]
    P_full = pd.concat([P_full,dt], axis=1).dropna()

end = time.process_time()
print('Running time: %s Seconds'%(end-start))

Running time: 0.703125 Seconds


In [46]:
P_full.Dates = P_full.Dates.apply(lambda x: pd.to_datetime(x, format='%Y%m%d'))
print(P_full.shape)
P_full.head()

(1952, 204)


,Dates,Portfolio_returns,Portfolio_returns_lag_1,Portfolio_flows_lag_1,ret_lag1_55976,flow_lag1_55976,ret_lag1_41072,flow_lag1_41072,ret_lag1_27748,flow_lag1_27748,...,ret_lag1_59379,flow_lag1_59379,ret_lag1_75162,flow_lag1_75162,ret_lag1_27705,flow_lag1_27705,ret_lag1_54594,flow_lag1_54594,ret_lag1_48725,flow_lag1_48725
3,1993-04-06,-0.003112,0.003637,-0.140873,0.040541,-6.975709,-0.005376,0.560749,0.000000,-1.436277,...,0.034274,12.213694,0.030864,0.297741,-0.011364,-4.991003,0.000000,-1.221645,0.001961,0.205041
4,1993-04-07,-0.001031,-0.003112,0.453289,0.038961,-3.351631,0.017568,-1.993177,0.004950,-3.149916,...,0.033138,1.779914,-0.005988,1.533468,-0.011494,-2.804367,0.020619,-4.795620,0.019569,0.819203
5,1993-04-08,-0.000309,-0.001031,-0.559098,-0.033333,-2.878666,-0.006640,-3.238632,0.002463,0.060249,...,0.009434,-11.525256,0.000000,-1.492312,0.000000,-2.508979,0.020202,1.846624,0.007678,1.384477
6,1993-04-12,0.009771,-0.000309,0.160661,-0.060345,-6.975709,0.014706,21.718050,0.000000,-1.271246,...,-0.042991,-18.556932,-0.018072,-1.696118,-0.034884,3.777336,0.009901,6.946835,-0.007619,4.387446
7,1993-04-13,0.003260,0.009771,1.440657,0.013761,-6.975709,0.011858,24.959539,-0.007371,1.099761,...,-0.025391,-14.936359,-0.006135,0.537455,0.012048,-0.818048,0.009804,3.727801,-0.013436,-0.159646
